In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
use_wandb = True

In [3]:
from ds_toolkit.general_utils.gpu_utils import gpu_alloc

gpu_alloc()

Setting proc title..
Process title :  MUHSIN_16-Nov_01-AM_NO_JOB_NAME
Searching for GPUs..
CUDA environment device set to 1
GPU allocation finished.


In [4]:
if use_wandb:
    import wandb
    wandb.login()

wandb: Currently logged in as: askmuhsin (use `wandb login --relogin` to force relogin)


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

import numpy as np
import matplotlib.pyplot as plt
from torchsummary import summary

In [6]:
from datasets.cifar10 import trainloader, testloader
from utils.general_utils import setup_env
from utils.training import train_model
from utils.testing import test_model

Files already downloaded and verified
Total number of images:  50000
Mean =  (0.4890062, 0.47970363, 0.47680542) 
 STD =  (0.264582, 0.258996, 0.25643882)
Files already downloaded and verified
Files already downloaded and verified
Batch Size -- 512


In [7]:
from models.model_v7 import Net

In [8]:
cuda, device = setup_env()

train_dataloader, test_dataloader = trainloader, testloader

model = Net(
    batch_norm=True,
    dropout_value=0.0,
).to(device)
summary(model, input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]             864
              ReLU-2           [-1, 32, 32, 32]               0
       BatchNorm2d-3           [-1, 32, 32, 32]              64
           Dropout-4           [-1, 32, 32, 32]               0
            Conv2d-5           [-1, 64, 32, 32]          18,432
              ReLU-6           [-1, 64, 32, 32]               0
       BatchNorm2d-7           [-1, 64, 32, 32]             128
           Dropout-8           [-1, 64, 32, 32]               0
        conv_block-9           [-1, 64, 32, 32]               0
           Conv2d-10           [-1, 32, 16, 16]          18,464
           Conv2d-11           [-1, 32, 16, 16]           1,056
             ReLU-12           [-1, 32, 16, 16]               0
      BatchNorm2d-13           [-1, 32, 16, 16]              64
 transition_block-14           [-1, 32,

In [12]:
if use_wandb:

    wandb.init(
        project='s7_cifar10',              ## unique to each project / assignment
        entity='weights_heist_eva7',       ## this will not change
        ## below are optional but recommended 
        tags=['muhsin'],                   ## can help later to filter runs
        name='version 7.2 continue',    ## a name for the run. 
        notes="continue training"
    )

wandb: wandb version 0.12.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [13]:
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
scheduler = StepLR(optimizer, step_size=12, gamma=0.6)

logs = []
EPOCHS = 40
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train_batch_loss, train_batch_acc = train_model(
        model, device, train_dataloader, optimizer, epoch
    )
    test_loss, test_acc = test_model(model, device, test_dataloader)
    scheduler.step()
    
    temp_log = {
        'train_losses': np.mean(train_batch_loss),
        'test_losses': test_loss,
        'train_acc': np.mean(train_batch_acc),
        'test_acc': test_acc,
        "lr": optimizer.param_groups[0]['lr'],
    }
    logs.append(temp_log)
    
    if use_wandb:
        wandb.log(temp_log)

EPOCH: 0


Loss=0.7177518010139465 Batch_id=97 Accuracy=77.45: 100%|██████████| 98/98 [00:09<00:00, 10.85it/s]



Test set: Average loss: 0.5912, Accuracy: 7989/10000 (79.89%)

EPOCH: 1


Loss=0.7636212110519409 Batch_id=97 Accuracy=77.57: 100%|██████████| 98/98 [00:09<00:00, 10.75it/s]



Test set: Average loss: 0.5593, Accuracy: 8106/10000 (81.06%)

EPOCH: 2


...


Test set: Average loss: 0.4240, Accuracy: 8615/10000 (86.15%)

EPOCH: 38


Loss=0.39051949977874756 Batch_id=97 Accuracy=85.00: 100%|██████████| 98/98 [00:08<00:00, 11.51it/s]



Test set: Average loss: 0.4218, Accuracy: 8613/10000 (86.13%)

EPOCH: 39


Loss=0.4365992844104767 Batch_id=97 Accuracy=85.42: 100%|██████████| 98/98 [00:08<00:00, 11.39it/s] 



Test set: Average loss: 0.4186, Accuracy: 8623/10000 (86.23%)



In [11]:
if use_wandb:
    wandb.finish()

train_losses,0.53289
test_losses,0.485
train_acc,81.18632
test_acc,83.52
lr,0.0216
_step,39
_runtime,415
_timestamp,1636996799


train_losses,█▆▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_losses,██▇▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▃▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇████████████████
test_acc,▁▂▃▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇████████████████
lr,███████████▄▄▄▄▄▄▄▄▄▄▄▄▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
